In [ ]:
# Import packages

import numpy as np    
import matplotlib.pyplot as plt       
from torchvision.transforms import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
from fastai.vision.all import *
set_seed(42, reproducible= True)
source = untar_data(URLs.IMAGENETTE)

In [ ]:
classes = ("Tench", "English Springer", "Cassette Player", "Chain Saw", "Church", "French Horn", "Garbage Truck", "Gas Pump", "Golf Ball", "Parachute")
batch_size = 64
width = 224
mean = [0.4655, 0.4546, 0.4251]
std = [0.2775, 0.2725, 0.2938]
# mean = [0.5,0.5,0.5]
# std = [0.27,0.27,0.27]


def load_data():
  train = source/"train"
  val = source/"val"

  train_dataset = ImageFolder(
    train,
    transforms.Compose([
        transforms.Resize(width),
        transforms.RandomCrop(width), 
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
        transforms.RandomErasing()
    ]))

  val_dataset = ImageFolder(
    val,
    transforms.Compose([
        transforms.Resize(width),
        transforms.RandomCrop(width), 
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]))

  train_dataloader = DataLoader(train_dataset, batch_size, shuffle=True)
  val_dataloader = DataLoader(val_dataset, batch_size)
  
  return train_dataloader, val_dataloader

In [ ]:
""" 
    ResNet50D based on 'Bag of Tricks for Image Classification with Convolutional Neural Networks' 
    (https://openaccess.thecvf.com/content_CVPR_2019/papers/He_Bag_of_Tricks_for_Image_Classification_with_Convolutional_Neural_Networks_CVPR_2019_paper.pdf) 
    modified from https://github.com/JayPatwardhan/ResNet-PyTorch/blob/master/ResNet/ResNet.py
"""
class Bottleneck(nn.Module):
    expansion = 4
    def __init__(self, in_channels, out_channels, i_downsample=None, stride=1):
        super(Bottleneck, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0)
        self.batch_norm1 = nn.BatchNorm2d(out_channels)
        
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.batch_norm2 = nn.BatchNorm2d(out_channels)
        
        self.conv3 = nn.Conv2d(out_channels, out_channels*self.expansion, kernel_size=1, stride=1, padding=0)
        self.batch_norm3 = nn.BatchNorm2d(out_channels*self.expansion)
        
        self.i_downsample = i_downsample
        self.stride = stride
        
    def forward(self, x):
        identity = x.clone()
        x = F.relu(self.batch_norm1(self.conv1(x)))
        
        x = F.relu(self.batch_norm2(self.conv2(x)))
        
        x = self.conv3(x)
        x = self.batch_norm3(x)
        
        if self.i_downsample is not None:
            identity = self.i_downsample(identity)
        
        x += identity
        x = F.relu(x)
        
        return x

class ResNet50D(nn.Module):
    def __init__(self, block):
        super(ResNet50D, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.batch_norm1 = nn.BatchNorm2d(64)
        self.max_pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(block, 3, planes=64)
        self.layer2 = self._make_layer(block, 4, planes=128, stride=2)
        self.layer3 = self._make_layer(block, 6, planes=256, stride=2)
        self.layer4 = self._make_layer(block, 3, planes=512, stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512*block.expansion, 10)
        
    def forward(self, x):
        x = F.relu(self.batch_norm1(self.conv1(x)))
        x = self.max_pool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        
        return x
        
    def _make_layer(self, ResBlock, blocks, planes, stride=1):
        ii_downsample = None
        layers = []
        
        if stride != 1:
          ii_downsample = nn.Sequential(
                nn.AvgPool2d(2, stride=2),
                nn.Conv2d(self.in_channels, planes*ResBlock.expansion, kernel_size=1, stride=1),
                nn.BatchNorm2d(planes*ResBlock.expansion)
            )
        elif self.in_channels != planes*ResBlock.expansion:
            ii_downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, planes*ResBlock.expansion, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes*ResBlock.expansion)
            )
            
        layers.append(ResBlock(self.in_channels, planes, i_downsample=ii_downsample, stride=stride))
        self.in_channels = planes*ResBlock.expansion
        
        for i in range(blocks-1):
            layers.append(ResBlock(self.in_channels, planes))
            
        return nn.Sequential(*layers)

In [ ]:
from torch.autograd import Variable
from torch.optim import Adam

# Function to save the model
def saveModel(cnn, p):
    path = "./"+p
    torch.save(cnn.state_dict(), path)

# Function to test the model with the test dataset and print the accuracy for the test images
def testAccuracy(cnn, device):
    
    cnn.eval()
    accuracy = 0.0
    total = 0.0
    
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images = Variable(images.to(device))
            labels = Variable(labels.to(device))
            # run the model on the test set to predict labels
            outputs = cnn(images)
            # the label with the highest energy will be our prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            accuracy += (predicted == labels).sum().item()
    
    # compute the accuracy over all test images
    accuracy = (100 * accuracy / total)
    return(accuracy)

def trainAccuracy(cnn, device):

    cnn.eval()
    accuracy = 0.0
    total = 0.0
    
    with torch.no_grad():
        for data in train_loader:
            images, labels = data
            images = Variable(images.to(device))
            labels = Variable(labels.to(device))
            # run the model on the test set to predict labels
            outputs = cnn(images)
            # the label with the highest energy will be our prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            accuracy += (predicted == labels).sum().item()
    
    # compute the accuracy over all test images
    accuracy = (100 * accuracy / total)
    return(accuracy)


# Training function. We simply have to loop over our data iterator and feed the inputs to the network and optimize.
def train(cnn, num_epochs, path):
    
    best_accuracy = 0.0

    # Define your execution device
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("The model will be running on", device, "device")
    # Convert model parameters and buffers to CPU or Cuda
    cnn.to(device)
 
    # Define the loss function with Classification Cross-Entropy loss and an optimizer with Adam optimizer
    loss_fn = nn.CrossEntropyLoss()
    optimizer = Adam(cnn.parameters(), lr=0.001, weight_decay=0.0001)

    all_accuracy = []

    for epoch in range(num_epochs):  # loop over the dataset multiple times
        running_loss = 0.0

        for i, (images, labels) in enumerate(train_loader, 0):
            # get the inputs
            images = Variable(images.to(device))
            labels = Variable(labels.to(device))

            # zero the parameter gradients
            optimizer.zero_grad()
            # predict classes using images from the training set
            outputs = cnn(images)
            # compute the loss based on model output and real labels
            loss = loss_fn(outputs, labels)
            # backpropagate the loss
            loss.backward()
            # adjust parameters based on the calculated gradients
            optimizer.step()

            # Let's print statistics for every 1000 images
            running_loss += loss.item()     # extract the loss value
            if i % 1000 == 999:    
                # print every 50 (twice per epoch) 
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 1000))
                # zero the loss
                running_loss = 0.0

        # Compute and print the average accuracy for this epoch when tested over all test images
        accuracy = testAccuracy(cnn, device)
        train_accuracy = trainAccuracy(cnn, device)
        all_accuracy.append(accuracy)
        print('For epoch', epoch+1,'the train accuracy is %d %%' % (train_accuracy), 'the test accuracy over the whole test set is %d %%' % (accuracy))
        
        # we want to save the model if the accuracy is the best
        if accuracy > best_accuracy:
            saveModel(cnn, path)
            best_accuracy = accuracy

In [5]:
# Let's build our model
train_loader, test_loader = load_data()

train(ResNet50D(Bottleneck), 80, 'classifier_ResNet50D.pth')

The model will be running on cuda:0 device
For epoch 1 the train accuracy is 43 % the test accuracy over the whole test set is 44 %
For epoch 2 the train accuracy is 26 % the test accuracy over the whole test set is 25 %
For epoch 3 the train accuracy is 35 % the test accuracy over the whole test set is 34 %
For epoch 4 the train accuracy is 38 % the test accuracy over the whole test set is 38 %
For epoch 5 the train accuracy is 43 % the test accuracy over the whole test set is 43 %
For epoch 6 the train accuracy is 49 % the test accuracy over the whole test set is 49 %
For epoch 7 the train accuracy is 52 % the test accuracy over the whole test set is 52 %
For epoch 8 the train accuracy is 58 % the test accuracy over the whole test set is 58 %
For epoch 9 the train accuracy is 58 % the test accuracy over the whole test set is 58 %
For epoch 10 the train accuracy is 63 % the test accuracy over the whole test set is 64 %
For epoch 11 the train accuracy is 63 % the test accuracy over the